# A simple flask webapp to try out our NER based heuristic function that returns news similarity

In [4]:
from flask import Flask
from flask import request
from flask import redirect

#iinstall spacy in your conda conda install -c conda-forge spacy
# python -m spacy download en_core_web_sm
import spacy
import os
import pandas as pd
import glob
from collections import defaultdict
import re
from datetime import datetime
from datetime import timedelta
import dateutil.parser
import pandas as pd

In [6]:
# https://spacy.io/api/annotation#named-entities

Weights = {'CARDINAL': 1,
 'DATE': 1,
 'EVENT': 1,
 'FAC':  2,
 'GPE': 2,
 'LANGUAGE': 1,
 'LAW': 2,
 'LOC': 2,
 'MONEY': 4,
 'NORP': 1,
 'ORDINAL': 1,
 'ORG': 16,
 'PERCENT': 16,
 'PERSON': 16,
 'PRODUCT': 4,
 'QUANTITY': 2,
 'TIME': 1,
 'WORK_OF_ART': 4
          }

# Load English tokenizer, tagger, parser, NER and word vectors
# nlp = spacy.load("en_core_web_lg")
# nlp = spacy.load("en_core_web_md")
nlp = spacy.load("en_core_web_sm")


def intersection(x, y):
    r = set.intersection(x, y).difference(['Join Livemint', 'Telegram', 'Mint'])
    return r


def weighted_score(inter):
    l = list(inter)
    score = 0
    for i in l:
        score += Weights[i.split(':')[0]]
    return score


def vec_similarity(x, y):
    if x is not None and y is not None:
        return x.similarity(y)

    return -1


def tag(x):
    s = set()
    doc = nlp(x)
    for ent in doc.ents:
        s.add(ent.label_+":"+ent.text)
    return s



def heuristic(x, y): 
    intersect = intersection(tag(x), tag(y))
    score = weighted_score(intersect)
    vec_sim = vec_similarity(nlp(x), nlp(y))
    verdict = 0
    if (vec_sim>0.95) and len(intersect)>2 and (score>100):
        verdict = 1
    r = "Score: "+str(score)+ "<br>Intersect: "+str(intersect)+"<br>Content Vector Similarity: "+str(vec_sim)+ "<br>Verdict: "+ str(verdict)
    return verdict, r

In [ ]:
app = Flask(__name__)

def get_index():
    return "<form action=\"/eval\" method=\"get\" id=\"eval\"> \
      <textarea name=\"x\" rows=\"20\" cols=\"100\" form=\"eval\">Enter text X here...</textarea> \
      <textarea name=\"y\" rows=\"20\" cols=\"100\" form=\"eval\">Enter text Y here...</textarea> \
      <input type=\"submit\">"
    

@app.route("/")
def init():
    return get_index()


@app.route("/eval")
def eval():
    x =request.args.get('x')
    y =request.args.get('y')
    verdict, r = heuristic(x, y)
    return r


if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Dec/2020 00:31:56] "GET /eval?x=%22The+Pradhan+Mantri+Jan+Dhan+Yojana+%28PMJDY%29%2C+the+countryâ€™s+financial+inclusion+initiative+that+was+launched+six+years+ago%2C+has+more+than+403.5+million+accounts+with+total+deposits+in+excess+of+Rs+1.30+lakh+crore%2C+the+Union+finance+ministry+said.%0D%0AThe+scheme%2C+which+has+become+the+basis+of+direct+transfer+of+cash+subsidy+to+the+poor%2C+was+announced+by+Prime+Minister+Narendra+Modi+in+his+Independence+Day+speech+on+August+15%2C+2014%2C+and+was+launched+on+August+28+that+year.%0D%0Aâ€œToday%2C+six+years+ago%2C+the+PMJDY+was+launched+with+an+ambitious+aim+of+banking+the+unbanked.+This+initiative+has+been+a+game-changer%2C+serving+as+the+foundation+for+several+poverty+alleviation+initiatives%2C+benefitting+crores+of+people%2Câ€+PM+Modi+tweeted+on+Friday.%0D%0AThe+government+is+planning+to+expand+the+scheme+to+provide+insurance+coverage+to+PMJDY+account+holders.+â€œ

127.0.0.1 - - [07/Dec/2020 00:34:52] "GET /eval?x=%22Six+ministers+from+Maharashtra%2C+Punjab%2C+West+Bengal%2C+Jharkhand%2C+Rajasthan+and+Chhattisgarh+moved+the+Supreme+Court+on+Friday+and+sought+review+of+its+August+17+order+allowing+conduct+of+NEET+%28UG%29+and+JEE+%28Mains%29+entrance+examinations.%0D%0AThe+plea+referred+to+the+rise+in+Covid-19+cases+in+the+country+and+said+that+if+the+August+17+order+is+not+reviewed%2C+grave+and+irreparable+harm+and+injury+would+befall+on+the+student+community.+The+petitioners+said+not+only+will+health%2C+welfare+and+safety+of+students%2Fcandidates+appearing+for+the+NEET%2FJEE+examinations+stand+imperilled+but+also+the+public+health+at+large+would+be+in+severe+jeopardyâ€¦%0D%0AThe+petitioners+are+Moloy+Ghatak+%28minister+from+West+Bengal%29%2C+Rameshwar+Oraon+%28Jharkhand%29%2C+Raghu+Sharma+%28Rajasthan%29%2C+Amarjeet+Bhagat+%28Chhattisgarh%29%2C+Balbir+Singh+Sidhu+%28Punjab%29%2C+and+Uday+Ravindra+Samant+%28Maharashtra%29.%0D%0AThe+petition%2C+fi

127.0.0.1 - - [07/Dec/2020 00:37:41] "GET /eval?x=%22Six+ministers+from+Maharashtra%2C+Punjab%2C+West+Bengal%2C+Jharkhand%2C+Rajasthan+and+Chhattisgarh+moved+the+Supreme+Court+on+Friday+and+sought+review+of+its+August+17+order+allowing+conduct+of+NEET+%28UG%29+and+JEE+%28Mains%29+entrance+examinations.%0D%0AThe+plea+referred+to+the+rise+in+Covid-19+cases+in+the+country+and+said+that+if+the+August+17+order+is+not+reviewed%2C+grave+and+irreparable+harm+and+injury+would+befall+on+the+student+community.+The+petitioners+said+not+only+will+health%2C+welfare+and+safety+of+students%2Fcandidates+appearing+for+the+NEET%2FJEE+examinations+stand+imperilled+but+also+the+public+health+at+large+would+be+in+severe+jeopardyâ€¦%0D%0AThe+petitioners+are+Moloy+Ghatak+%28minister+from+West+Bengal%29%2C+Rameshwar+Oraon+%28Jharkhand%29%2C+Raghu+Sharma+%28Rajasthan%29%2C+Amarjeet+Bhagat+%28Chhattisgarh%29%2C+Balbir+Singh+Sidhu+%28Punjab%29%2C+and+Uday+Ravindra+Samant+%28Maharashtra%29.%0D%0AThe+petition%2C+fi

127.0.0.1 - - [07/Dec/2020 00:44:08] "GET /eval?x=At+08%3A45+AM%2C+Nifty+futures+on+the+Singapore+Exchange+%28SGX%29+traded+4.5+points+or+0.04+per+cent+higher+at+11%2C569.20%2C+indicating+a+flat+start+for+the+Indian+market+on+Thursday.+Here%27s+a+list+of+stocks+that+may+trade+actively+in+today%27s+trading+session.+Coal+India%3A+Coal+India+Ltd+%28CIL%29+on+Wednesday+posted+55+per+cent+decline+in+consolidated+net+profit+at+Rs+2%2C079.60+crore+for+the+quarter+ended+June+30+on+the+back+of+lower+sales.+Jubilant+FoodWorks%3A+Jubilant+FoodWorks+Ltd%2C+which+operates+fast-food+chains+Domino%27s+Pizza+and+Dunkin%27+Donuts%2C+on+Wednesday+reported+a+consolidated+net+loss+of+Rs+74.47+crore+for+the+first+quarter+ended+June+2020%2C+impacted+by+closure+of+stores+due+to+Covid-19+pandemic+and+consequent+lockdown.+Banks%2C+NBFCs%3A+Finance+Minister+Nirmala+Sitharaman+will+hold+a+review+meeting+with+heads+of+banks+and+NBFCs+on+Thursday+for+smooth+and+speedy+implementation+of+the+one-time+debt+recast+for

In [10]:
def confusion(row):
    suffix = 'N'
    if int(row.prediction) == 1:
        suffix = 'P'
    prefix = 'F'
    if row.prediction == row['binary-label']:
        prefix = 'T'
    return prefix+suffix

no_of = lambda x: len(result[result['confusion']==x])

def print_quality(result):
    result['prediction'] = result.apply(lambda row: on_row(row), axis=1)
    result['confusion'] = result.apply(lambda row: confusion(row), axis = 1)
    Accuracy = (no_of('TP') + no_of('TN'))/(no_of('TP') + no_of('TN') + no_of('FP') + no_of('FN'))
    Precision = no_of('TP') / (no_of('TP') + no_of('FP'))
    Recall = no_of('TP') / (no_of('TP') + no_of('FN'))
    print('Accuracy = '+ str(Accuracy))
    print('Precision = '+ str(Precision))
    print('Recall = '+ str(Recall))

def on_row(row):
    x = row['content_x']
    y = row['content_y']
    verdict, r = heuristic(x, y)
    return verdict

test_data = 'D:\\newsapibackup\\benchmark-data_balance.csv'
# test_data = 'D:\\newsapibackup\\benchmark-data_real.csv'
result = pd.read_csv(test_data)


Sorry reader. I need to suppress the warnings. They will be printed in a loop otherwise

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
print_quality(result)

Accuracy = 0.8
Precision = 0.7380952380952381
Recall = 0.6966292134831461
